In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [2]:
path = r"C:\Program Files (x86)\chromedriver.exe"
resume = r"C:\Users\pleasework\Documents\DataResume2020.pdf"

In [3]:
## Things to add:
## applying to multiple positions at once
## checking requirements
## keep track of jobs that are not easy apply (company website)

# Goals: Weed out jobs before switching tabs (requirements)

In [4]:
def apply(job, location, name, email, phone, resume):
    driver = webdriver.Chrome(path)
    #Link to indeed home
    driver.get("https://www.indeed.com/?from=gnav-jobsearch--jasx")
    jobtitle = driver.find_element_by_name("q")
    jobtitle.send_keys(job)
    where = driver.find_element_by_name("l")
    #Clears the default location on indeed
    #hacky solution - maxlen allowed for place is 62, so just press backspace 62 times lol
    #alternative solution - select all, then backspace in the field
    for _ in range(62):
        where.send_keys(Keys.BACKSPACE);
    where.send_keys(location)
    where.send_keys(Keys.RETURN)
    
    ####Applying####
    cards = driver.find_elements_by_class_name("jobsearch-SerpJobCard")
    job_dates = driver.find_elements_by_class_name("date")
    # New list of jobs if the job was posted less than 20 days ago
    new_jobs = [job for job in job_dates if int(float(job.text[0:2])) < 20]
    # Remove cards based on the indices where day posted was over 20 days ago
    new_jobs = set(new_jobs)
    indices = [index for index, job in enumerate(job_dates) if job in new_jobs]
    new_cards = [cards[index] for index in indices]
    for card in new_cards:
        # only click the ones with indeed easy apply
        if "Easily apply" in card.get_attribute('innerHTML'):
            link = card.find_element_by_xpath(".//a[@class='jobtitle turnstileLink ']")
            actions = ActionChains(driver)
            actions.key_down(Keys.COMMAND).click(link).key_up(Keys.CONTROL).perform()
            driver.switch_to.window(driver.window_handles[-1])
            apply = driver.find_elements_by_tag_name("button")
            apply[2].click()
            html = driver.page_source
            iframe = driver.find_elements_by_tag_name("iframe")
            framenamesrc = [(i.get_attribute('name'), i.get_attribute('src')) for i in iframe]
            for framename, src in framenamesrc:
                if framename.startswith('indeedapply'):
                    driver.switch_to.frame(framename)
                    iframe = driver.find_elements_by_tag_name("iframe")
                    iframeinner = driver.find_elements_by_tag_name("iframe")
                    srcinner = [i.get_attribute('src') for i in iframeinner]
                    for innersrc in srcinner:
                        if innersrc.startswith('https://apply.indeed.com/indeedapply/'):
                            print(innersrc)
                            driver.get(innersrc)
                            break
                    break              
            namefield = driver.find_element_by_id("input-applicant.name")
            namefield.send_keys("")
            namefield.send_keys(name)
            emailfield = driver.find_element_by_id("input-applicant.email")
            emailfield.send_keys(email)
            phonefield = driver.find_element_by_id("input-applicant.phoneNumber")
            phonefield.send_keys(phone)
            resumefield = driver.find_element_by_id("ia-FilePicker-resume")
            resumefield.send_keys(resume)
            driver.switch_to.window(driver.window_handles[0])
    time.sleep(3)
    driver.quit()

In [8]:
def apply(job, location, name, email, phone, resume):
    driver = webdriver.Chrome(path)
    #Link to indeed home
    driver.get("https://www.indeed.com/?from=gnav-jobsearch--jasx")
    jobtitle = driver.find_element_by_name("q")
    jobtitle.send_keys(job)
    where = driver.find_element_by_name("l")
    #Clears the default location on indeed
    #hacky solution - maxlen allowed for place is 62, so just press backspace 62 times lol
    #alternative solution - select all, then backspace in the field
    for _ in range(62):
        where.send_keys(Keys.BACKSPACE);
    where.send_keys(location)
    where.send_keys(Keys.RETURN)
    
    ####Applying####
    cards = driver.find_elements_by_class_name("jobsearch-SerpJobCard")
    for card in cards:
        # only click the ones with indeed easy apply
        if "Easily apply" in card.get_attribute('innerHTML'):
            link = card.find_element_by_xpath(".//a[@class='jobtitle turnstileLink ']")
            actions = ActionChains(driver)
            actions.key_down(Keys.COMMAND).click(link).key_up(Keys.CONTROL).perform()
            driver.switch_to.window(driver.window_handles[-1])
            apply = driver.find_elements_by_tag_name("button")
            apply[2].click()
            html = driver.page_source
            iframe = driver.find_elements_by_tag_name("iframe")
            framenamesrc = [(i.get_attribute('name'), i.get_attribute('src')) for i in iframe]
            for framename, src in framenamesrc:
                if framename.startswith('indeedapply'):
                    driver.switch_to.frame(framename)
                    iframe = driver.find_elements_by_tag_name("iframe")
                    iframeinner = driver.find_elements_by_tag_name("iframe")
                    srcinner = [i.get_attribute('src') for i in iframeinner]
                    for innersrc in srcinner:
                        if innersrc.startswith('https://apply.indeed.com/indeedapply/'):
                            print(innersrc)
                            driver.get(innersrc)
                            break
                    break        

In [ ]:
                    # Check the iframe for the words "Bachelors" or "BS", if not present qualifications don't fit
                    jobDetails = driver.page_source
                    jobDetails = jobDetails.lower()
                    if ("bachelor" or "bs") not in jobDetails:
                        continue

In [5]:
apply("Python Entry Developer", "Boston, Massachusetts", "First Last", "emailgmail@gmail.com", "1234567890", resume)